<center><img src="https://storage.googleapis.com/unskript-website/assets/favicon.png" alt="unSkript.com" width="100" height="100">
<h1 id="unSkript-Runbooks&para;">unSkript Runbooks</h1>
<div class="alert alert-block alert-success">
<h3 id="Objective&para;">Objective</h3>
<br><strong style="color: #000000;"><em><strong>Restrict S3 Buckets with READ/WRITE Permissions for all authenticated users.</strong></em></strong></div>
</center>
<p>&nbsp;</p>
<center>
<h2 id="Restrict-S3-Buckets-with-READ/WRITE-Permissions&para;"><u>Restrict S3 Buckets with READ/WRITE Permissions</u></h2>
</center>
<h1 id="Steps-Overview&para;">Steps Overview</h1>
<p>1)&nbsp;<a href="#1" target="_self" rel="noopener">Filter Public S3 buckets with ACL Permissions</a><br>2)&nbsp;<a href="#2" target="_self" rel="noopener">Change the permissions to private</a></p>

In [3]:
if region == None:
    region = ''
if bucket_names and not region:
    raise SystemExit("Provide a region for the S3 bucket names!")

<h3 id="List-expiring-ACM-certificates"><a id="1" target="_self" rel="nofollow"></a>Filter S3 buckets with ACL Permissiosn<a class="jp-InternalAnchorLink" href="#List-expiring-ACM-certificates" target="_self">&para;</a></h3>
<p>This action will fetch all public S3 buckets with the chosen permissions- <em>"READ","READ_ACP","WRITE","WRITE_ACP", and "FULL_CONTROL"</em>, If no permissions are given, the action will execute for <span style="color: blue;"> READ</span> and <span style="color: blue;"> WRITE</span>.</p>
<blockquote>
<p>This action takes the following parameters: <code>bucket_permission(Optional)</code>, <code>region(Optional)</code></p>
</blockquote>
<blockquote>
<p>This action captures the following output: <code>public_buckets</code></p>
</blockquote>

In [4]:
##
##  Copyright (c) 2021 unSkript, Inc
##  All rights reserved.
##
from pydantic import BaseModel, Field
from typing import List, Optional, Tuple
from unskript.legos.aws.aws_list_all_regions.aws_list_all_regions import aws_list_all_regions
from unskript.legos.aws.aws_get_s3_buckets.aws_get_s3_buckets import aws_get_s3_buckets
from unskript.enums.aws_acl_permissions_enums import BucketACLPermissions
import pprint


from beartype import beartype
@beartype
def aws_filter_public_s3_buckets_by_acl_printer(output):
    if output is None:
        return
    pprint.pprint(output)

@beartype
def check_publicly_accessible_buckets(s3Client,b,all_permissions):
    public_check = ["http://acs.amazonaws.com/groups/global/AuthenticatedUsers",
                   "http://acs.amazonaws.com/groups/global/AllUsers"]
    public_buckets = False
    try:
        res = s3Client.get_bucket_acl(Bucket=b)
        for perm in all_permissions:
            for grant in res["Grants"]:
                if 'Permission' in grant.keys() and perm == grant["Permission"]:
                    if 'URI' in grant["Grantee"] and grant["Grantee"]["URI"] in public_check:
                        public_buckets = True
    except Exception as e:
        pass
    return public_buckets

@beartype
def aws_filter_public_s3_buckets_by_acl(handle, permission:BucketACLPermissions=BucketACLPermissions.READ, region: str=None) -> Tuple:
    """aws_filter_public_s3_buckets_by_acl get list of public buckets.

        Note- By default(if no permissions are given) READ and WRITE ACL Permissioned S3 buckets are checked for public access. Other ACL Permissions are - "READ_ACP"|"WRITE_ACP"|"FULL_CONTROL"
        :type handle: object
        :param handle: Object returned from task.validate(...)

        :type permission: Enum
        :param permission: Set of permissions that AWS S3 supports in an ACL for buckets and objects.

        :type region: string
        :param region: location of the bucket.

        :rtype: Object with status, list of public S3 buckets with READ/WRITE ACL Permissions, and errors
    """
    all_permissions = [permission]
    if permission is None or len(permission)==0:
        all_permissions = ["READ","WRITE"]
    result = []
    all_buckets = []
    all_regions = [region]
    if region is None or len(region)==0:
        all_regions = aws_list_all_regions(handle=handle)
    try:
        for r in all_regions:
            s3Client = handle.client('s3',region_name=r)
            output = aws_get_s3_buckets(handle=handle, region=r)
            if len(output)!= 0:
                for o in output:
                    all_buckets_dict = {}
                    all_buckets_dict["region"]=r
                    all_buckets_dict["bucket"]=o
                    all_buckets.append(all_buckets_dict)
    except Exception as e:
        raise e

    for bucket in all_buckets:
        s3Client = handle.client('s3',region_name= bucket['region'])
        flag = check_publicly_accessible_buckets(s3Client,bucket['bucket'], all_permissions)
        if flag:
            result.append(bucket)
    if len(result)!=0:
        return (False, result)
    else:
        return (True, None)


task = Task(Workflow())
task.configure(inputParamsJson='''{
    "permission": "bucket_permission",
    "region": "\\"us-west-2\\""
    }''')
task.configure(conditionsJson='''{
    "condition_enabled": true,
    "condition_cfg": "not bucket_names",
    "condition_result": true
    }''')

task.configure(outputName="public_buckets")

task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_filter_public_s3_buckets_by_acl, lego_printer=aws_filter_public_s3_buckets_by_acl_printer, hdl=hdl, args=args)

<h3 id="Create-List-of-Expiring-Certificates"><a id="1" target="_self" rel="nofollow"></a>Create List of public S3 Buckets<a class="jp-InternalAnchorLink" href="#Create-List-of-Expiring-Certificates" target="_self">&para;</a></h3>
<p>This action filters regions that have no public buckets and creates a list of public buckets that have are to be made private.</p>
<blockquote>
<p>This action takes the following parameters: <code>None</code></p>
</blockquote>
<blockquote>
<p>This action captures the following output: <code>all_public_buckets</code></p>
</blockquote>

In [25]:
all_public_buckets = []
try:
    if public_buckets[0] == False:
        if len(public_buckets[1])!=0:
            all_public_buckets=public_buckets[1]
except Exception:
    for buck in bucket_names:
        data_dict = {}
        data_dict["region"] = region
        data_dict["bucket"] = buck
        all_public_buckets.append(data_dict)
print(all_public_buckets)

<h3 id="List-expiring-ACM-certificates"><a id="1" target="_self" rel="nofollow"></a>Change permission to private<a class="jp-InternalAnchorLink" href="#List-expiring-ACM-certificates" target="_self">&para;</a></h3>
<p>Using unSkript's AWS Change ACL Permission of public S3 Bucket action, we will fchange the permissions of the bucket to <em>private, public-read, public-read-write, authenticated-read.&nbsp;</em>If no canned_acl_permission is selected, <span style="color: blue;"> private</span> will be set by default.&nbsp;</p>
<blockquote>
<p>This action takes the following parameters: <code>bucket_name</code>, <code>region,canned_acl_permission</code></p>
</blockquote>
<blockquote>
<p>This action captures the following output: <code>None</code></p>
</blockquote>

In [ ]:
##
# Copyright (c) 2021 unSkript, Inc
# All rights reserved.
##
import pprint
from typing import Optional, Dict
from pydantic import BaseModel, Field
from unskript.enums.aws_canned_acl_enums import CannedACLPermissions



from beartype import beartype
@beartype
def aws_change_acl_permissions_of_buckets_printer(output):
    if output is None:
        return
    pprint.pprint(output)


@beartype
def aws_change_acl_permissions_of_buckets(
    handle,
    bucket_name: str,
    acl: CannedACLPermissions=CannedACLPermissions.Private,
    region: str = None
    ) -> Dict:
    """ aws_put_bucket_acl get Dict of buckets ACL change info.

            :type handle: Session
            :param handle: Object returned by the task.validate(...) method

            :type bucket_name: string
            :param bucket_name: S3 bucket name where to set ACL on.

            :type acl: CannedACLPermissions
            :param acl: Canned ACL Permission type - 'private'|'public-read'|'public-read-write
            '|'authenticated-read'.

            :type region: string
            :param region: location of the bucket.

            :rtype: Dict of buckets ACL change info
    """
    # connect to the S3 using client
    all_permissions = acl
    if acl is None or len(acl)==0:
        all_permissions = "private"
    s3Client = handle.client('s3',
                             region_name=region)

    # Put bucket ACL for the permissions grant
    response = s3Client.put_bucket_acl(
                    Bucket=bucket_name,
                    ACL=all_permissions )

    return response


task = Task(Workflow())
task.configure(continueOnError=True)
task.configure(inputParamsJson='''{
    "region": "\\"iter.get(\\\\\\"region\\\\\\")\\"",
    "bucket_name": "\\"iter.get(\\\\\\"bucket_name\\\\\\")\\"",
    "acl": "acl_permission"
    }''')
task.configure(iterJson='''{
    "iter_enabled": true,
    "iter_list_is_const": false,
    "iter_list": "all_public_buckets",
    "iter_parameter": ["region","bucket_name"]
    }''')

task.configure(conditionsJson='''{
    "condition_enabled": true,
    "condition_cfg": "len(all_public_buckets)!=0",
    "condition_result": true
    }''')

task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_change_acl_permissions_of_buckets, lego_printer=aws_change_acl_permissions_of_buckets_printer, hdl=hdl, args=args)

<h3 id="Conclusion">Conclusion<a class="jp-InternalAnchorLink" href="#Conclusion" target="_self">&para;</a></h3>
<p>In this Runbook, we were able to restrict S3 buckets having read and write permissions to private. To view the full platform capabilities of unSkript please visit&nbsp;<a href="https://us.app.unskript.io" target="_blank" rel="noopener">us.app.unskript.io</a></p>